In [2]:
import numpy as np
import cvxpy as cp
import itertools

Here is the problem, shown again for your convenience:


\begin{align*}
        \min\; &ax + by + c\\
        \text{subject to}\;\; &x \geq 0,\\
        & y \ge 0,\\
        &x + 2y \leq 6,\\
        &5x + 2y \leq 10.
    \end{align*}

### **Define the constraints**

Let the constraint be expressed as ax + by <= c
We will define the constraints in form of tuples, with the first element being [a,b] and second element being c 

In [3]:
constraints = []
constraints.append(([1,2],6))
#TODO Add other constraints here
constraints.append(([5,2],10))
constraints.append(([-1, 0], 0))
constraints.append(([0, -1], 0))

### **Define function to find corners of polyhedron formed by the constraints**

In [4]:
def findCorners(constraints):
  intersections = []
  for (con1,con2) in itertools.combinations(constraints,2):
    if np.linalg.det(np.stack([con1[0],con2[0]]))!=0:#non-parallel lines
      a = np.array([con1[0], con2[0]])
      b = np.array([con1[1], con2[1]])
      x,y = np.linalg.solve(a, b)
      intersections.append((x,y))

  corners = intersections.copy()
  for point in intersections:
    point_inside = np.prod([con[0]@np.array(point)<=con[1] for con in constraints])
    if not point_inside: corners.remove(point)

  return corners

### For the following optimization objectives, **Find the value of the objective at the corner points** and **deduce the solution of the optimization problem** from the values at the corner points. Express the objective in the form of ax+by+c, and obj as [a,b,c]

### **Objective = -2x+3y+5**

In [8]:
#TODO find the corners of polytope
corners = findCorners(constraints)
obj = [-2,3,5]
for corner in corners:
  objVal = np.array(obj[:2]).T @ np.array(corner) + obj[2]
  print("Corner Point:",corner, "Objective value:", objVal)

Corner Point: (1.0, 2.5) Objective value: 10.5
Corner Point: (0.0, 3.0) Objective value: 14.0
Corner Point: (2.0, -0.0) Objective value: 1.0
Corner Point: (-0.0, -0.0) Objective value: 5.0


***TODO: Enter the solution(s) for above optimization problem and your justifications here***

**Answer:** The solution is 1.0 (achieved at (2, 0)) since this is the minimum objective value at the corners.

### **Objective = -x-2y+5**

In [9]:
corners = findCorners(constraints)
obj = [-1,-2,5]
for corner in corners:
  objVal = np.array(obj[:2]).T @ np.array(corner) + obj[2]
  print("Corner Point:",corner, "Objective value:", objVal)

Corner Point: (1.0, 2.5) Objective value: -1.0
Corner Point: (0.0, 3.0) Objective value: -1.0
Corner Point: (2.0, -0.0) Objective value: 3.0
Corner Point: (-0.0, -0.0) Objective value: 5.0


***TODO: Enter the solution(s) for above optimization problem and your justifications here***

**Answer:** The solution is -1.0 (at both (1, 2.5), and (0, 3)) as this is the minimum objective value at the corners.

### **Objective = 5**

In [12]:
corners = findCorners(constraints)
obj = 5
for corner in corners:
  objVal = obj
  print("Corner Point:",corner, "Objective value:", objVal)

Corner Point: (1.0, 2.5) Objective value: 5
Corner Point: (0.0, 3.0) Objective value: 5
Corner Point: (2.0, -0.0) Objective value: 5
Corner Point: (-0.0, -0.0) Objective value: 5


***TODO: Enter the solution(s) for above optimization problem and your justifications here***

**Answer:** The objective value is always 5. It is achieved at any feasible point, for example (1,2.5).

### **Now we will solve the problems using an optimization tool cvxpy** 

### **Define the variable**

In [14]:
x = cp.Variable(2)

### **Define the constraints**

In [15]:
constraints = []
#TODO Add constraints to the list, following cvxpy syntax for constraints
constraints.append(x[0]+2*x[1]<=6)
constraints.append(5*x[0]+2*x[1]<=10)
constraints.append(-x[0]<=0)
constraints.append(-x[1]<=0)

### **Run optimization for various objectives**

In [16]:
objectives = [[-2,3,5],[-1,-2,5],[0,0,5]]
for obj in objectives:
  #TODO formulate the objective
  objective = cp.Minimize(obj[0]*x[0]+obj[1]*x[1]+obj[2])
  #Define the optimization problem in cvxpy syntax
  prob = cp.Problem(objective, constraints)
  prob.solve()
  print("\nObjective = %dx+%dy+%d. The optimal value is %.2f and a solution x is"%(obj[0],obj[1],obj[2],prob.value),np.round(x.value,1))


Objective = -2x+3y+5. The optimal value is 1.00 and a solution x is [2. 0.]

Objective = -1x+-2y+5. The optimal value is -1.00 and a solution x is [0.6 2.7]

Objective = 0x+0y+5. The optimal value is 5.00 and a solution x is [0.9 1.6]


/usr/local/lib/python3.10/site-packages/cvxpy/reductions/solvers/solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)
